# **Objective:**

In this assignment, you will implement a Decision Tree Classifier from
scratch using numpy. and apply it to the Adult Income Dataset. The
task is to predict whether a person earns more than $50K per year. You will
build the tree, evaluate it, and perform both pre-pruning and post-pruning.

# **Dataset**

We will use the *Adult Income Dataset* from UCI.

• Dataset link: Adult Dataset

• Task: Binary classification (≤ 50K vs. > 50K).

• Features: Mix of categorical (e.g., workclass, education, occupation)
and numeric (e.g., age, hours-per-week).

• Target: Income.

# Loading Instructions

Use the following code to fetch the dataset:

pip italicized text install ucimlrepo

from ucimlrepo import fetch_ucirepo

adult = fetch_ucirepo(id=2)

X = adult.data.features # features (pandas DataFrame)

y = adult.data.targets # target (pandas DataFrame)

In [52]:
!pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier as SklearnDT

# **Instructions**

Follow these steps carefully. Do not skip any part.

**1. Data Preparation**

• Handle missing values (drop or impute).

• Encode categorical variables into numeric values (e.g., label encoding).

• Split the dataset as:

– 80% training

– 20% validation

– 20% test

Use the validation set to tune depth and pruning.

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

data = pd.read_csv(url, names=columns, sep=',\s*', engine='python', na_values='?')
data = data.dropna()  # Remove rows with missing values

# Features and target
X = data.drop(columns=['income'])
y = data['income']

# Encode categorical features
for col in X.select_dtypes('object').columns:
    X[col] = LabelEncoder().fit_transform(X[col])

y = LabelEncoder().fit_transform(y)

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-332368444.py:8: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(url, names=columns, sep=',\s*', engine='python', na_values='?')


In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)
print('Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)

Train: (18096, 14) Val: (6033, 14) Test: (6033, 14)


**2. Build a Decision Tree From Scratch**

• Implement a tree recursively.

• At each split:

1. Compute both Gini Impurity and Entropy.

2. For each feature and split, calculate the weighted impurity of child
nodes.

3. Choose the split with the highest information gain (lowest impu-
rity).

• Continue splitting until:

– All samples in a node have the same label, OR

– Maximum depth is reached, OR

– No further improvement in impurity.

• Implement a function to predict for new samples.

In [ ]:
# Decision Tree from Scratch
class TreeNode:
    def __init__(self, depth=0):
        self.feature_idx = None
        self.threshold = None
        self.left = None
        self.right = None
        self.is_leaf = False
        self.pred_label = None
        self.depth = depth

class DecisionTreeCustom:
    def __init__(self, criterion='gini', max_depth=None, min_samples_split=2):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def _gini_index(self, y):
        probs = np.bincount(y) / len(y)
        return 1 - np.sum(probs ** 2)

    def _entropy(self, y):
        probs = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in probs if p > 0])

    def _impurity(self, y):
        return self._gini_index(y) if self.criterion == 'gini' else self._entropy(y)

    def _find_best_split(self, X, y):
        best_gain, best_feature, best_thresh = 0, None, None
        parent_impurity = self._impurity(y)
        for col in range(X.shape[1]):
            unique_vals = np.unique(X[:, col])
            if len(unique_vals) == 1:
                continue
            thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2
            for t in thresholds:
                left_y = y[X[:, col] <= t]
                right_y = y[X[:, col] > t]
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                weighted_impurity = (len(left_y) * self._impurity(left_y) + len(right_y) * self._impurity(right_y)) / len(y)
                gain = parent_impurity - weighted_impurity
                if gain > best_gain:
                    best_gain, best_feature, best_thresh = gain, col, t
        return best_feature, best_thresh, best_gain

    def _build_tree(self, X, y, depth):
        node = TreeNode(depth)
        if len(np.unique(y)) == 1 or (self.max_depth and depth >= self.max_depth) or len(y) < self.min_samples_split:
            node.is_leaf = True
            node.pred_label = Counter(y).most_common(1)[0][0]
            return node

        feat, thr, gain = self._find_best_split(X, y)
        if feat is None:
            node.is_leaf = True
            node.pred_label = Counter(y).most_common(1)[0][0]
            return node

        node.feature_idx = feat
        node.threshold = thr
        left_mask = X[:, feat] <= thr
        right_mask = X[:, feat] > thr
        node.left = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        node.right = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        return node

    def fit(self, X, y):
        self.root = self._build_tree(np.array(X), np.array(y), 0)

    def _predict_sample(self, x, node):
        if node.is_leaf:
            return node.pred_label
        if x[node.feature_idx] <= node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

    def predict(self, X):
        return np.array([self._predict_sample(x, self.root) for x in np.array(X)])

**3. Pre-Pruning (Restricting Tree Growth)**

While building the tree:

• Limit maximum depth (try depths = 2, 4, 6, and unlimited).

• Require at least a minimum number of samples (e.g., 5) to split.

• Require a minimum impurity decrease (optional).

**4. Post-Pruning (Reduced Error Pruning)**

1. First grow a full tree.

2. Then, for each internal node:

• Replace it with a leaf (majority class).

• Check validation accuracy.

3. If accuracy does not decrease, keep the pruning.

4. Repeat until no further improvement.

In [ ]:
# Reduced Error Pruning
def reduced_error_pruning(tree, X_val, y_val):
    best_acc = accuracy_score(y_val, tree.predict(X_val))
    improvement = True

    def internal_nodes(node):
        nodes_list = []
        def traverse(n):
            if n and not n.is_leaf:
                nodes_list.append(n)
                traverse(n.left)
                traverse(n.right)
        traverse(node)
        return nodes_list

    while improvement:
        improvement = False
        nodes = internal_nodes(tree.root)
        for n in nodes:
            backup = (n.feature_idx, n.threshold, n.left, n.right, n.is_leaf, n.pred_label)
            n.is_leaf = True
            n.left = n.right = None
            n.pred_label = Counter(y_train).most_common(1)[0][0]
            new_acc = accuracy_score(y_val, tree.predict(X_val))
            if new_acc >= best_acc:
                best_acc = new_acc
                improvement = True
            else:
                n.feature_idx, n.threshold, n.left, n.right, n.is_leaf, n.pred_label = backup

In [ ]:
# Train & Evaluate
model_custom = DecisionTreeCustom(criterion='gini', max_depth=6, min_samples_split=5)
model_custom.fit(X_train, y_train)

print('Train Accuracy:', accuracy_score(y_train, model_custom.predict(X_train)))
print('Val Accuracy:', accuracy_score(y_val, model_custom.predict(X_val)))

print('Validation accuracy before pruning:', accuracy_score(y_val, model_custom.predict(X_val)))
reduced_error_pruning(model_custom, X_val, y_val)
print('Validation accuracy after pruning:', accuracy_score(y_val, model_custom.predict(X_val)))

Train Accuracy: 0.8545534924845269
Val Accuracy: 0.8506547323056523
Validation accuracy before pruning: 0.8506547323056523
Validation accuracy after pruning: 0.8511519973479198


**5. Evaluation**

• Train using the training set.

• Tune depth and pruning using validation set.

• Report final results on test set.

• Metrics to report:

– Accuracy

– Precision, Recall, F1-score

– Confusion Matrix

• Compare your implementation with sklearn.tree.DecisionTreeClassifier.

In [ ]:
# Evaluating model
def evaluate_model(model, X, y):
    preds = model.predict(X)
    print("Accuracy:", accuracy_score(y, preds))
    print("Precision:", precision_score(y, preds, average='weighted'))
    print("Recall:", recall_score(y, preds, average='weighted'))
    print("F1 Score:", f1_score(y, preds, average='weighted'))
    print("Confusion Matrix:\n", confusion_matrix(y, preds))

In [ ]:
# Comparing different depths
for d in [2, 4, 6, None]:
    print(f"\n Depth = {d}")
    m = DecisionTreeCustom(max_depth=d, min_samples_split=5)
    m.fit(X_train, y_train)
    evaluate_model(m, X_val, y_val)


 Depth = 2
Accuracy: 0.8219791148682247
Precision: 0.8138132099089248
Recall: 0.8219791148682247
F1 Score: 0.8023114355673042
Confusion Matrix:
 [[4332  199]
 [ 875  627]]

 Depth = 4
Accuracy: 0.8478368970661363
Precision: 0.8414250499153328
Recall: 0.8478368970661363
F1 Score: 0.8381950700834553
Confusion Matrix:
 [[4294  237]
 [ 681  821]]

 Depth = 6
Accuracy: 0.8506547323056523
Precision: 0.8442326744157831
Recall: 0.8506547323056523
F1 Score: 0.8429707254503235
Confusion Matrix:
 [[4268  263]
 [ 638  864]]

 Depth = None
Accuracy: 0.8092159787833582
Precision: 0.8114496068372945
Recall: 0.8092159787833582
F1 Score: 0.810268552844933
Confusion Matrix:
 [[3930  601]
 [ 550  952]]


**6. Experiments to Perform**

• Compare Gini vs. Entropy.

• Compare different depths (2, 4, 6, unlimited).

• Show effect of pruning (pre-pruned vs. post-pruned vs. full tree).

• Identify the most important features (which features are used at the
top of the tree).

In [ ]:
# Compare criteria
for c in ['gini', 'entropy']:
    print(f"\n Criterion = {c} ")
    m = DecisionTreeCustom(criterion=c, max_depth=6)
    m.fit(X_train, y_train)
    evaluate_model(m, X_val, y_val)


 Criterion = gini 
Accuracy: 0.8508204873197415
Precision: 0.8444173850022059
Recall: 0.8508204873197415
F1 Score: 0.8431698246308788
Confusion Matrix:
 [[4268  263]
 [ 637  865]]

 Criterion = entropy 
Accuracy: 0.8508204873197415
Precision: 0.8452931458381515
Recall: 0.8508204873197415
F1 Score: 0.840608636904044
Confusion Matrix:
 [[4317  214]
 [ 686  816]]


In [ ]:
# Sklearn comparison
sk_model = SklearnDT(criterion='gini', max_depth=6, random_state=42)
sk_model.fit(X_train, y_train)
print('Sklearn Test Accuracy:', accuracy_score(y_test, sk_model.predict(X_test)))
print('Scratch Test Accuracy:', accuracy_score(y_test, model_custom.predict(X_test)))

Sklearn Test Accuracy: 0.8470081219956904
Scratch Test Accuracy: 0.8468423669816012
